In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
import os

In [3]:
os.chdir('C:\\Users\\Dell Notebook\\Desktop\\internship\\prediction_of_hitting_goal_or_not-master\\datasets')

In [4]:
dataset=pd.read_csv("train.csv")

In [5]:
dataset.head()

,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,remaining_sec,distance_of_shot,is_goal,...,lat/lng,type_of_shot,type_of_combined_shot,match_id,team_id,remaining_min.1,power_of_shot.1,knockout_match.1,remaining_sec.1,distance_of_shot.1
0,12.0,-157.0,0.0,10.0,1.0,0.0,2000-01,22.0,35.0,0.0,...,"45.539131, -122.651648",shot - 45,NaN,20000012,1610612747,10.00,1.0,28.800,22.0000,35.0
1,35.0,-101.0,135.0,7.0,1.0,0.0,2000-01,45.0,36.0,1.0,...,"45.539131, -122.651648",shot - 25,NaN,20000012,1610612747,92.64,1.0,0.000,63.7216,54.4
2,43.0,138.0,175.0,6.0,1.0,0.0,2000-01,52.0,42.0,0.0,...,"45.539131, -122.651648",NaN,shot - 3,20000012,1610612747,NaN,1.0,122.608,52.0000,42.0
3,155.0,0.0,0.0,NaN,2.0,0.0,2000-01,19.0,20.0,1.0,...,"45.539131, -122.651648",NaN,shot - 1,20000012,1610612747,42.64,2.0,0.000,19.0000,20.0
4,244.0,-145.0,-11.0,9.0,3.0,0.0,NaN,32.0,34.0,0.0,...,"45.539131, -122.651648",shot - 17,NaN,20000012,1610612747,9.00,3.0,0.000,NaN,34.0


In [6]:
dataset.isna().sum()

match_event_id            979
location_x                903
location_y                991
remaining_min            1000
power_of_shot             931
knockout_match            934
game_season              3688
remaining_sec            1012
distance_of_shot         1019
is_goal                     0
area_of_shot              941
shot_basics              1045
range_of_shot            1010
team_name                 964
date_of_game              994
home/away                 982
shot_id_number              0
lat/lng                   999
type_of_shot             9758
type_of_combined_shot    9846
match_id                    0
team_id                     0
remaining_min.1           967
power_of_shot.1           978
knockout_match.1          966
remaining_sec.1           968
distance_of_shot.1       1013
dtype: int64

In [7]:
dataset.drop(["match_event_id"],axis=1,inplace=True)

In [8]:
dataset.drop(["team_name"],axis=1,inplace=True)

In [9]:
dataset.drop(["date_of_game"],axis=1,inplace=True)

In [10]:
dataset.drop(["shot_id_number"],axis=1,inplace=True)

In [11]:
dataset.drop(["match_id"],axis=1,inplace=True)

In [12]:
dataset.drop(["team_id"],axis=1,inplace=True)

In [13]:
dataset.isnull().sum()

location_x                903
location_y                991
remaining_min            1000
power_of_shot             931
knockout_match            934
game_season              3688
remaining_sec            1012
distance_of_shot         1019
is_goal                     0
area_of_shot              941
shot_basics              1045
range_of_shot            1010
home/away                 982
lat/lng                   999
type_of_shot             9758
type_of_combined_shot    9846
remaining_min.1           967
power_of_shot.1           978
knockout_match.1          966
remaining_sec.1           968
distance_of_shot.1       1013
dtype: int64

In [14]:
print(dataset["location_x"].max())
print(dataset["location_x"].min())
print(dataset["location_x"].mean())
print(dataset["location_x"].var())
print(dataset["location_x"].mode())

248.0
-250.0
7.889791989733169
12261.283308090535
0    0.0
dtype: float64


In [15]:
dataset["location_x"]=dataset["location_x"].fillna(0.0)

In [16]:
print(dataset["location_y"].max())
print(dataset["location_y"].min())
print(dataset["location_y"].mean())
print(dataset["location_y"].var())
print(dataset["location_y"].mode())

741.0
-44.0
91.31848707892334
7657.160327133156
0    0.0
dtype: float64


In [17]:
dataset["location_y"]=dataset["location_y"].fillna(0.0)

In [18]:
print(dataset["remaining_min"].max())
print(dataset["remaining_min"].min())
print(dataset["remaining_min"].mean())
print(dataset["remaining_min"].var())
print(dataset["remaining_min"].mode())

11.0
0.0
4.864168995914857
11.866675001487929
0    0.0
dtype: float64


In [19]:
temp=dataset["remaining_min"].mean()
dataset["remaining_min"]=dataset["remaining_min"].fillna(temp)

In [20]:
print(dataset["power_of_shot"].max())
print(dataset["power_of_shot"].min())
print(dataset["power_of_shot"].mean())
print(dataset["power_of_shot"].var())
print(dataset["power_of_shot"].mode())

7.0
1.0
2.5203234616826435
1.33100563658387
0    3.0
dtype: float64


In [21]:
temp=dataset["power_of_shot"].mean()
dataset["power_of_shot"]=dataset["power_of_shot"].fillna(temp)

In [22]:
print(dataset["knockout_match"].max())
print(dataset["knockout_match"].min())
print(dataset["knockout_match"].mean())
print(dataset["knockout_match"].var())
print(dataset["knockout_match"].mode())

0.0
0.0
0.0
0.0
0    0.0
dtype: float64


In [23]:
dataset["knockout_match"]=dataset["knockout_match"].fillna(0.0)

In [24]:
dataset["game_season"].mode()

0    2005-06
dtype: object

In [25]:
dataset["game_season"]=dataset["game_season"].fillna("2005-06")

In [26]:
print(dataset["remaining_sec"].max())
print(dataset["remaining_sec"].min())
print(dataset["remaining_sec"].mean())
print(dataset["remaining_sec"].var())
print(dataset["remaining_sec"].mode())

59.0
0.0
28.269685886402755
305.7836477309149
0    0.0
dtype: float64


In [27]:
dataset["remaining_sec"]=dataset["remaining_sec"].fillna(0.0)

In [28]:
print(dataset["distance_of_shot"].max())
print(dataset["distance_of_shot"].min())
print(dataset["distance_of_shot"].mean())
print(dataset["distance_of_shot"].var())
print(dataset["distance_of_shot"].mode())

94.0
20.0
33.50486951842884
87.55821186551108
0    20.0
dtype: float64


In [29]:
dataset["distance_of_shot"]=dataset["distance_of_shot"].fillna(20.0)

In [30]:
print(dataset["area_of_shot"].mode())

0    Center(C)
dtype: object


In [31]:
dataset["area_of_shot"]=dataset["area_of_shot"].fillna("Center(C)")

In [32]:
print(dataset["shot_basics"].mode())

0    Mid Range
dtype: object


In [33]:
dataset["shot_basics"]=dataset["shot_basics"].fillna("Mid Range")

In [34]:
print(dataset["range_of_shot"].mode())

0    Less Than 8 ft.
dtype: object


In [35]:
dataset["range_of_shot"]=dataset["range_of_shot"].fillna("Less Than 8 ft.")

In [36]:
print(dataset["home/away"].mode())

0    MANU @ SAC
dtype: object


In [37]:
dataset["home/away"]=dataset["home/away"].fillna("MANU @ SAC")

In [38]:
print(dataset["lat/lng"].mode())

0    42.982923, -71.446094
dtype: object


In [39]:
dataset["lat/lng"]=dataset["lat/lng"].fillna("42.982923, -71.446094")

In [40]:
print(dataset["type_of_shot"].mode())

0    shot - 39
dtype: object


In [41]:
dataset["type_of_shot"]=dataset["type_of_shot"].fillna("shot - 39")

In [42]:
print(dataset["type_of_combined_shot"].mode())

0    shot - 3
dtype: object


In [43]:
dataset["type_of_combined_shot"]=dataset["type_of_combined_shot"].fillna("shot - 3")

In [44]:
print(dataset["remaining_min.1"].max())
print(dataset["remaining_min.1"].min())
print(dataset["remaining_min.1"].mean())
print(dataset["remaining_min.1"].var())
print(dataset["remaining_min.1"].mode())

128.7616
0.0
18.025788528198156
863.3297688239046
0    0.0
dtype: float64


In [45]:
dataset["remaining_min.1"]=dataset["remaining_min.1"].fillna(0.0)

In [46]:
print(dataset["power_of_shot.1"].max())
print(dataset["power_of_shot.1"].min())
print(dataset["power_of_shot.1"].mean())
print(dataset["power_of_shot.1"].var())
print(dataset["power_of_shot.1"].mode())

118.36
1.0
15.986760442390935
879.6450555235225
0    3.0
dtype: float64


In [47]:
temp=dataset["power_of_shot.1"].mode()
dataset["power_of_shot.1"]=dataset["power_of_shot.1"].fillna(3.0)

In [48]:
print(dataset["knockout_match.1"].max())
print(dataset["knockout_match.1"].min())
print(dataset["knockout_match.1"].mean())
print(dataset["knockout_match.1"].var())
print(dataset["knockout_match.1"].mode())

141.35232000000005
0.0
16.33176505633686
1230.5122009758836
0    0.0
dtype: float64


In [49]:
tmp=dataset["knockout_match.1"].mode()
dataset["knockout_match.1"]=dataset["knockout_match.1"].fillna(0.0)

In [50]:
print(dataset["remaining_sec.1"].max())
print(dataset["remaining_sec.1"].min())
print(dataset["remaining_sec.1"].mean())
print(dataset["remaining_sec.1"].var())
print(dataset["remaining_sec.1"].mode())

144.7856
0.0
38.794915132000746
884.1592675255725
0    0.0
dtype: float64


In [51]:
tmp=dataset["remaining_sec.1"].mode()
dataset["remaining_sec.1"]=dataset["remaining_sec.1"].fillna(0.0)

In [52]:
print(dataset["distance_of_shot.1"].max())
print(dataset["distance_of_shot.1"].min())
print(dataset["distance_of_shot.1"].mean())
print(dataset["distance_of_shot.1"].var())
print(dataset["distance_of_shot.1"].mode())

115.728
9.4
38.76825775913298
346.87591471266313
0    20.0
dtype: float64


In [53]:
tp=dataset["distance_of_shot.1"].mode()
dataset["distance_of_shot.1"]=dataset["distance_of_shot.1"].fillna(20.0)

In [54]:
dataset

,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,remaining_sec,distance_of_shot,is_goal,area_of_shot,...,range_of_shot,home/away,lat/lng,type_of_shot,type_of_combined_shot,remaining_min.1,power_of_shot.1,knockout_match.1,remaining_sec.1,distance_of_shot.1
0,-157.0,0.0,10.000000,1.0,0.0,2000-01,22.0,35.0,0.0,Left Side(L),...,8-16 ft.,MANU @ POR,"45.539131, -122.651648",shot - 45,shot - 3,10.0000,1.00,28.800,22.0000,35.0
1,-101.0,135.0,7.000000,1.0,0.0,2000-01,45.0,36.0,1.0,Left Side Center(LC),...,16-24 ft.,MANU @ SAC,"45.539131, -122.651648",shot - 25,shot - 3,92.6400,1.00,0.000,63.7216,54.4
2,138.0,175.0,6.000000,1.0,0.0,2000-01,52.0,42.0,0.0,Right Side Center(RC),...,16-24 ft.,MANU @ POR,"45.539131, -122.651648",shot - 39,shot - 3,0.0000,1.00,122.608,52.0000,42.0
3,0.0,0.0,4.864169,2.0,0.0,2000-01,19.0,20.0,1.0,Center(C),...,Less Than 8 ft.,MANU @ POR,"45.539131, -122.651648",shot - 39,shot - 1,42.6400,2.00,0.000,19.0000,20.0
4,-145.0,-11.0,9.000000,3.0,0.0,2005-06,32.0,34.0,0.0,Left Side(L),...,8-16 ft.,MANU @ POR,"45.539131, -122.651648",shot - 17,shot - 3,9.0000,3.00,0.000,0.0000,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19599,-169.0,121.0,10.000000,2.0,0.0,1999-00,41.0,40.0,0.0,Left Side(L),...,16-24 ft.,MANU @ SAC,"38.567296, -121.456638",shot - 39,shot - 3,10.0000,2.00,0.000,41.0000,40.0
19600,0.0,0.0,5.000000,2.0,0.0,1999-00,23.0,20.0,1.0,Center(C),...,Less Than 8 ft.,MANU @ SAC,"38.567296, -121.456638",shot - 39,shot - 5,38.7616,41.36,119.928,71.7856,95.4
19601,-162.0,3.0,4.000000,2.0,0.0,1999-00,34.0,36.0,1.0,Left Side(L),...,Less Than 8 ft.,MANU @ SAC,"38.567296, -121.456638",shot - 56,shot - 3,23.6400,2.00,0.000,34.0000,36.0
19602,0.0,0.0,2.000000,2.0,0.0,1999-00,26.0,20.0,1.0,Center(C),...,Less Than 8 ft.,MANU @ SAC,"38.567296, -121.456638",shot - 39,shot - 4,2.0000,2.00,49.928,0.0000,20.0


In [55]:
dataset.isnull().sum()

location_x               0
location_y               0
remaining_min            0
power_of_shot            0
knockout_match           0
game_season              0
remaining_sec            0
distance_of_shot         0
is_goal                  0
area_of_shot             0
shot_basics              0
range_of_shot            0
home/away                0
lat/lng                  0
type_of_shot             0
type_of_combined_shot    0
remaining_min.1          0
power_of_shot.1          0
knockout_match.1         0
remaining_sec.1          0
distance_of_shot.1       0
dtype: int64

In [56]:
dataset["remaining_min"]=(dataset["remaining_min"]+dataset["remaining_min.1"])/2

In [57]:
dataset.drop(["remaining_min.1"],axis=1,inplace=True)

In [58]:
dataset["power_of_shot"]=(dataset["power_of_shot"]+dataset["power_of_shot.1"])/2
dataset.drop(["power_of_shot.1"],axis=1,inplace=True)

In [59]:
dataset["knockout_match"]=(dataset["knockout_match"]+dataset["knockout_match.1"])/2
dataset.drop(["knockout_match.1"],axis=1,inplace=True)

In [60]:
dataset["remaining_sec"]=(dataset["remaining_sec"]+dataset["remaining_sec.1"])/2
dataset.drop(["remaining_sec.1"],axis=1,inplace=True)

In [61]:
dataset["distance_of_shot"]=(dataset["distance_of_shot"]+dataset["distance_of_shot.1"])/2
dataset.drop(["distance_of_shot.1"],axis=1,inplace=True)

In [62]:
dataset.isnull().sum()

location_x               0
location_y               0
remaining_min            0
power_of_shot            0
knockout_match           0
game_season              0
remaining_sec            0
distance_of_shot         0
is_goal                  0
area_of_shot             0
shot_basics              0
range_of_shot            0
home/away                0
lat/lng                  0
type_of_shot             0
type_of_combined_shot    0
dtype: int64

In [63]:
tpp=dataset["is_goal"]

In [64]:
dataset.drop(["is_goal"],axis=1,inplace=True)

In [65]:
dataset["is_goal"]=tpp

In [66]:
dataset.isnull().sum()

location_x               0
location_y               0
remaining_min            0
power_of_shot            0
knockout_match           0
game_season              0
remaining_sec            0
distance_of_shot         0
area_of_shot             0
shot_basics              0
range_of_shot            0
home/away                0
lat/lng                  0
type_of_shot             0
type_of_combined_shot    0
is_goal                  0
dtype: int64

In [67]:
ttt=dataset["game_season"]
dataset.drop(["game_season"],axis=1,inplace=True)
dataset["game_season"]=ttt

In [68]:
dataset.isnull().sum()

location_x               0
location_y               0
remaining_min            0
power_of_shot            0
knockout_match           0
remaining_sec            0
distance_of_shot         0
area_of_shot             0
shot_basics              0
range_of_shot            0
home/away                0
lat/lng                  0
type_of_shot             0
type_of_combined_shot    0
is_goal                  0
game_season              0
dtype: int64

In [69]:
ha=dataset["home/away"]
dataset.drop(["home/away"],axis=1,inplace=True)
dataset["home/away"]=ha
laln=dataset["lat/lng"]
dataset.drop(["lat/lng"],axis=1,inplace=True)
dataset["lat/lng"]=laln
ggss=dataset["game_season"]
dataset.drop(["game_season"],axis=1,inplace=True)
dataset["game_season"]=ggss
iigg=dataset["is_goal"]
dataset.drop(["is_goal"],axis=1,inplace=True)
dataset["is_goal"]=iigg

In [70]:
dataset.isnull().sum()

location_x               0
location_y               0
remaining_min            0
power_of_shot            0
knockout_match           0
remaining_sec            0
distance_of_shot         0
area_of_shot             0
shot_basics              0
range_of_shot            0
type_of_shot             0
type_of_combined_shot    0
home/away                0
lat/lng                  0
game_season              0
is_goal                  0
dtype: int64

In [71]:
dataset.to_csv("train_clean.csv",index=False)

In [72]:
dd=pd.read_csv("test.csv")

In [73]:
dd.isnull().sum()

match_event_id            189
location_x                194
location_y                189
remaining_min             183
power_of_shot             200
knockout_match            200
game_season               708
remaining_sec             176
distance_of_shot          171
area_of_shot              173
shot_basics               170
range_of_shot             176
team_name                 172
date_of_game              181
home/away                 160
shot_id_number              0
lat/lng                   167
type_of_shot             1758
type_of_combined_shot    1817
match_id                    0
team_id                     0
remaining_min.1           164
power_of_shot.1           163
knockout_match.1          193
remaining_sec.1           174
distance_of_shot.1        163
dtype: int64

In [74]:
print(dd["location_x"].max())
print(dd["location_x"].min())
print(dd["location_x"].mean())
print(dd["location_x"].var())
print(dd["location_x"].mode())

244.0
-245.0
5.3356994971901806
11530.413012151028
0    0.0
dtype: float64


In [75]:
dd["location_x"]=dd["location_x"].fillna(0.0)

In [76]:
dd.drop(["match_event_id"],axis=1,inplace=True)

In [77]:
dd.drop(["team_name"],axis=1,inplace=True)

In [78]:
dd.drop(["date_of_game"],axis=1,inplace=True)

In [79]:
dd.drop(["match_id"],axis=1,inplace=True)
dd.drop(["team_id"],axis=1,inplace=True)

In [80]:
dd.drop(["shot_id_number"],axis=1,inplace=True)
dd.isnull().sum()

location_x                  0
location_y                189
remaining_min             183
power_of_shot             200
knockout_match            200
game_season               708
remaining_sec             176
distance_of_shot          171
area_of_shot              173
shot_basics               170
range_of_shot             176
home/away                 160
lat/lng                   167
type_of_shot             1758
type_of_combined_shot    1817
remaining_min.1           164
power_of_shot.1           163
knockout_match.1          193
remaining_sec.1           174
distance_of_shot.1        163
dtype: int64

In [81]:
print(dd["location_x"].max())
print(dd["location_x"].min())
print(dd["location_x"].mean())
print(dd["location_x"].var())
print(dd["location_x"].mode())

244.0
-245.0
5.046153846153846
10905.993112651253
0    0.0
dtype: float64


In [82]:
dd["location_x"]=dd["location_x"].fillna(0.0)

In [83]:
print(dd["remaining_min"].max())
print(dd["remaining_min"].min())
print(dd["remaining_min"].mean())
print(dd["remaining_min"].var())
print(dd["remaining_min"].mode())

11.0
0.0
4.956073113207547
12.195946592116737
0    0.0
dtype: float64


In [84]:
dd["remaining_min"]=dd["remaining_min"].fillna(12.195)

In [85]:
print(dd["power_of_shot"].max())
print(dd["power_of_shot"].min())
print(dd["power_of_shot"].mean())
print(dd["power_of_shot"].var())
print(dd["power_of_shot"].mode())

5.0
1.0
2.5303703703703704
1.3238402599398367
0    3.0
dtype: float64


In [86]:
dd["power_of_shot"]=dd["power_of_shot"].fillna(1.32)

In [87]:
print(dd["knockout_match"].max())
print(dd["knockout_match"].min())
print(dd["knockout_match"].mean())
print(dd["knockout_match"].var())
print(dd["knockout_match"].mode())

1.0
0.0
0.9588148148148148
0.039500669608546694
0    1.0
dtype: float64


In [88]:
dd["knockout_match"]=dd["knockout_match"].fillna(0.958)

In [89]:
print(dd["game_season"].mode())

0    1999-00
dtype: object


In [90]:
dd["game_season"]=dd["game_season"].fillna("1999-00")

In [91]:
print(dd["remaining_sec"].max())
print(dd["remaining_sec"].min())
print(dd["remaining_sec"].mean())
print(dd["remaining_sec"].var())
print(dd["remaining_sec"].mode())

59.0
0.0
28.075022065313327
307.6709444889053
0    0.0
dtype: float64


In [92]:
dd["remaining_sec"]=dd["remaining_sec"].fillna(0.0)

In [93]:
print(dd["distance_of_shot"].max())
print(dd["distance_of_shot"].min())
print(dd["distance_of_shot"].mean())
print(dd["distance_of_shot"].var())
print(dd["distance_of_shot"].mode())

99.0
20.0
33.21357226792009
88.38017424661315
0    20.0
dtype: float64


In [94]:
dd["distance_of_shot"]=dd["distance_of_shot"].fillna(20.0)

In [95]:
print(dd["area_of_shot"].mode())

0    Center(C)
dtype: object


In [96]:
dd["area_of_shot"]=dd["area_of_shot"].fillna("Center(C)")

In [97]:
print(dd["shot_basics"].mode())

0    Mid Range
dtype: object


In [98]:
dd["shot_basics"]=dd["shot_basics"].fillna("Mid Range")

In [99]:

print(dd["range_of_shot"].mode())

0    Less Than 8 ft.
dtype: object


In [100]:
dd["range_of_shot"]=dd["range_of_shot"].fillna("Less Than 8 ft.")

In [101]:
print(dd["home/away"].mode())

0    MANU @ SAS
dtype: object


In [102]:
dd["home/away"]=dd["home/away"].fillna("MANU @ SAS")

In [103]:
print(dd["lat/lng"].mode())

0    42.982923, -71.446094
dtype: object


In [104]:
dd["lat/lng"]=dd["lat/lng"].fillna("42.982923, -71.446094")

In [105]:
print(dd["type_of_shot"].mode())

0    shot - 39
dtype: object


In [106]:
dd["type_of_shot"]=dd["type_of_shot"].fillna("shot - 39")

In [107]:
print(dd["type_of_combined_shot"].mode())

0    shot - 3
dtype: object


In [108]:
dd["type_of_combined_shot"]=dd["type_of_combined_shot"].fillna("shot - 3")

In [109]:
print(dd["remaining_min.1"].max())
print(dd["remaining_min.1"].min())
print(dd["remaining_min.1"].mean())
print(dd["remaining_min.1"].var())
print(dd["remaining_min.1"].mode())

119.64
0.0
18.87421870419218
886.436898902312
0    0.0
dtype: float64


In [110]:
dd["remaining_min.1"]=dd["remaining_min.1"].fillna(0.0)

In [111]:
print(dd["power_of_shot.1"].max())
print(dd["power_of_shot.1"].min())
print(dd["power_of_shot.1"].mean())
print(dd["power_of_shot.1"].var())
print(dd["power_of_shot.1"].mode())

118.36
1.0
16.354607268464324
895.4706106050533
0    1.0
dtype: float64


In [112]:
dd["power_of_shot.1"]=dd["power_of_shot.1"].fillna(1.0)

In [113]:
print(dd["knockout_match.1"].max())
print(dd["knockout_match.1"].min())
print(dd["knockout_match.1"].mean())
print(dd["knockout_match.1"].var())
print(dd["knockout_match.1"].mode())

137.608
0.0
17.38781019515087
1220.4108590412973
0    1.0
dtype: float64


In [114]:
dd["knockout_match.1"]=dd["knockout_match.1"].fillna(1.0)

In [115]:
print(dd["remaining_sec.1"].max())
print(dd["remaining_sec.1"].min())
print(dd["remaining_sec.1"].mean())
print(dd["remaining_sec.1"].var())
print(dd["remaining_sec.1"].mode())

144.7856
0.0
38.88159623640105
888.1805875561282
0    0.0
dtype: float64


In [116]:
dd["remaining_sec.1"]=dd["remaining_sec.1"].fillna(0.0)

In [117]:
print(dd["distance_of_shot.1"].max())
print(dd["distance_of_shot.1"].min())
print(dd["distance_of_shot.1"].mean())
print(dd["distance_of_shot.1"].var())
print(dd["distance_of_shot.1"].mode())

115.728
9.4
38.75886518171125
366.476508938726
0    20.0
dtype: float64


In [118]:
dd["distance_of_shot.1"]=dd["distance_of_shot.1"].fillna(20.0)

In [119]:
dd.isnull().sum()

location_x                 0
location_y               189
remaining_min              0
power_of_shot              0
knockout_match             0
game_season                0
remaining_sec              0
distance_of_shot           0
area_of_shot               0
shot_basics                0
range_of_shot              0
home/away                  0
lat/lng                    0
type_of_shot               0
type_of_combined_shot      0
remaining_min.1            0
power_of_shot.1            0
knockout_match.1           0
remaining_sec.1            0
distance_of_shot.1         0
dtype: int64

In [120]:
print(dd["location_y"].max())
print(dd["location_y"].min())
print(dd["location_y"].mean())
print(dd["location_y"].var())
print(dd["location_y"].mode())

791.0
-42.0
91.19728292971057
8119.702870015727
0    0.0
dtype: float64


In [121]:
dd["location_y"]=dd["location_y"].fillna(0.0)

In [122]:
dd.isna().sum()

location_x               0
location_y               0
remaining_min            0
power_of_shot            0
knockout_match           0
game_season              0
remaining_sec            0
distance_of_shot         0
area_of_shot             0
shot_basics              0
range_of_shot            0
home/away                0
lat/lng                  0
type_of_shot             0
type_of_combined_shot    0
remaining_min.1          0
power_of_shot.1          0
knockout_match.1         0
remaining_sec.1          0
distance_of_shot.1       0
dtype: int64

In [123]:
dd["remaining_min"]=(dd["remaining_min"]+dd["remaining_min.1"])/2
dd.drop(["remaining_min.1"],axis=1,inplace=True)

In [124]:
dd["power_of_shot"]=(dd["power_of_shot"]+dd["power_of_shot.1"])/2
dd.drop(["power_of_shot.1"],axis=1,inplace=True)

In [125]:
dd["knockout_match"]=(dd["knockout_match"]+dd["knockout_match.1"])/2
dd.drop(["knockout_match.1"],axis=1,inplace=True)

In [126]:
dd["remaining_sec"]=(dd["remaining_sec"]+dd["remaining_sec.1"])/2
dd.drop(["remaining_sec.1"],axis=1,inplace=True)

In [127]:
dd["distance_of_shot"]=(dd["distance_of_shot"]+dd["distance_of_shot.1"])/2
dd.drop(["distance_of_shot.1"],axis=1,inplace=True)

In [128]:
dd.isnull().sum()

location_x               0
location_y               0
remaining_min            0
power_of_shot            0
knockout_match           0
game_season              0
remaining_sec            0
distance_of_shot         0
area_of_shot             0
shot_basics              0
range_of_shot            0
home/away                0
lat/lng                  0
type_of_shot             0
type_of_combined_shot    0
dtype: int64

In [129]:
dataset.isnull().sum()

location_x               0
location_y               0
remaining_min            0
power_of_shot            0
knockout_match           0
remaining_sec            0
distance_of_shot         0
area_of_shot             0
shot_basics              0
range_of_shot            0
type_of_shot             0
type_of_combined_shot    0
home/away                0
lat/lng                  0
game_season              0
is_goal                  0
dtype: int64

In [130]:
hoaw=dd["home/away"]
dd.drop(["home/away"],axis=1,inplace=True)
dd["home/away"]=hoaw
lat=dd["lat/lng"]
dd.drop(["lat/lng"],axis=1,inplace=True)
dd["lat/lng"]=lat

In [131]:
ggs=dd["game_season"]
dd.drop(["game_season"],axis=1,inplace=True)
dd["game_season"]=ggs

In [132]:
dd.isnull().sum()

location_x               0
location_y               0
remaining_min            0
power_of_shot            0
knockout_match           0
remaining_sec            0
distance_of_shot         0
area_of_shot             0
shot_basics              0
range_of_shot            0
type_of_shot             0
type_of_combined_shot    0
home/away                0
lat/lng                  0
game_season              0
dtype: int64

In [133]:
dataset.isnull().sum()

location_x               0
location_y               0
remaining_min            0
power_of_shot            0
knockout_match           0
remaining_sec            0
distance_of_shot         0
area_of_shot             0
shot_basics              0
range_of_shot            0
type_of_shot             0
type_of_combined_shot    0
home/away                0
lat/lng                  0
game_season              0
is_goal                  0
dtype: int64

In [134]:
dd.to_csv("test_clean.csv",index=False)

In [135]:
X_train=dataset.iloc[:,0:15]

In [136]:
Y_train=dataset.iloc[:,15]

In [137]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [138]:
X_test = np.array(dd)

In [139]:
dataset.drop(["is_goal"],axis=1,inplace=True)
big = pd.DataFrame()
big=pd.concat([dataset,dd], axis=0)

In [140]:
big.head()

,location_x,location_y,remaining_min,power_of_shot,knockout_match,remaining_sec,distance_of_shot,area_of_shot,shot_basics,range_of_shot,type_of_shot,type_of_combined_shot,home/away,lat/lng,game_season
0,-157.0,0.0,10.000000,1.0,14.400,22.0000,35.0,Left Side(L),Mid Range,8-16 ft.,shot - 45,shot - 3,MANU @ POR,"45.539131, -122.651648",2000-01
1,-101.0,135.0,49.820000,1.0,0.000,54.3608,45.2,Left Side Center(LC),Mid Range,16-24 ft.,shot - 25,shot - 3,MANU @ SAC,"45.539131, -122.651648",2000-01
2,138.0,175.0,3.000000,1.0,61.304,52.0000,42.0,Right Side Center(RC),Mid Range,16-24 ft.,shot - 39,shot - 3,MANU @ POR,"45.539131, -122.651648",2000-01
3,0.0,0.0,23.752084,2.0,0.000,19.0000,20.0,Center(C),Goal Area,Less Than 8 ft.,shot - 39,shot - 1,MANU @ POR,"45.539131, -122.651648",2000-01
4,-145.0,-11.0,9.000000,3.0,0.000,16.0000,34.0,Left Side(L),Mid Range,8-16 ft.,shot - 17,shot - 3,MANU @ POR,"45.539131, -122.651648",2005-06


In [141]:
X=big.iloc[:, 0:15].values
X

array([[-157.0, 0.0, 10.0, ..., 'MANU @ POR', '45.539131, -122.651648',
        '2000-01'],
       [-101.0, 135.0, 49.82, ..., 'MANU @ SAC',
        '45.539131, -122.651648', '2000-01'],
       [138.0, 175.0, 3.0, ..., 'MANU @ POR', '45.539131, -122.651648',
        '2000-01'],
       ...,
       [-134.0, 166.0, 3.0, ..., 'MANU vs. IND', '42.982923, -71.446094',
        '1999-00'],
       [31.0, 267.0, 2.0, ..., 'MANU vs. IND', '42.982923, -71.446094',
        '1999-00'],
       [1.0, 0.0, 0.0, ..., 'MANU vs. IND', '42.982923, -71.446094',
        '1999-00']], dtype=object)

In [142]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [143]:
labelencoder_X_7 = LabelEncoder()
labelencoder_X_8 = LabelEncoder()
labelencoder_X_9 = LabelEncoder()
labelencoder_X_10 = LabelEncoder()
labelencoder_X_11 = LabelEncoder()
labelencoder_X_12 = LabelEncoder()
labelencoder_X_13 = LabelEncoder()
labelencoder_X_14 = LabelEncoder()

In [144]:
X[:,7]

array(['Left Side(L)', 'Left Side Center(LC)', 'Right Side Center(RC)',
       ..., 'Left Side Center(LC)', 'Center(C)', 'Center(C)'],
      dtype=object)

In [145]:
X[:, 7] = labelencoder_X_7.fit_transform(X[:, 7])
X[:, 8] = labelencoder_X_8.fit_transform(X[:, 8])
X[:, 9] = labelencoder_X_9.fit_transform(X[:, 9])
X[:, 10] = labelencoder_X_10.fit_transform(X[:, 10])
X[:, 11] = labelencoder_X_11.fit_transform(X[:, 11])
X[:, 12] = labelencoder_X_12.fit_transform(X[:, 12])
X[:, 13] = labelencoder_X_13.fit_transform(X[:, 13])
X[:, 14] = labelencoder_X_14.fit_transform(X[:, 14])

In [146]:
tmpdf = pd.DataFrame(X)
tmpdf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-157,0,10,1,14.4,22,35,2,4,2,40,3,28,34,4
1,-101,135,49.82,1,0,54.3608,45.2,1,4,0,18,3,29,34,4
2,138,175,3,1,61.304,52,42,4,4,0,33,3,28,34,4
3,0,0,23.7521,2,0,19,20,0,0,4,33,1,28,34,4
4,-145,-11,9,3,0,16,34,2,4,2,9,3,28,34,9


In [147]:
onehotencoder=OneHotEncoder(categorical_features=[7,8,9,10,11,12,13,14])
X=onehotencoder.fit_transform(X).toarray()

C:\Users\Dell Notebook\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\Dell Notebook\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [148]:
tmpdf = pd.DataFrame(X)
tmpdf.head()

,0,1,2,3,4,5,6,7,8,9,...,210,211,212,213,214,215,216,217,218,219
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-157.0,0.0,10.000000,1.0,14.400,22.0000,35.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-101.0,135.0,49.820000,1.0,0.000,54.3608,45.2
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,138.0,175.0,3.000000,1.0,61.304,52.0000,42.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,23.752084,2.0,0.000,19.0000,20.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-145.0,-11.0,9.000000,3.0,0.000,16.0000,34.0


In [149]:
hahatrain=pd.DataFrame()
hahatest = pd.DataFrame()

In [150]:
hahatest=X[19604:,:]

In [151]:
hahatest

array([[ 1.   ,  0.   ,  0.   , ...,  0.   , 37.   , 20.   ],
       [ 0.   ,  0.   ,  1.   , ...,  0.   , 37.   , 32.   ],
       [ 1.   ,  0.   ,  0.   , ..., 21.804, 15.   , 20.   ],
       ...,
       [ 0.   ,  1.   ,  0.   , ...,  1.   , 28.   , 41.   ],
       [ 1.   ,  0.   ,  0.   , ...,  1.   , 10.   , 46.   ],
       [ 1.   ,  0.   ,  0.   , ...,  1.   , 39.   , 27.   ]])

In [152]:
hahatrain=X[0:19604,:]

In [153]:
from xgboost import XGBClassifier
classifier=XGBClassifier()
classifier.fit(hahatrain,Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [154]:
ypred=classifier.predict(hahatest)

In [155]:
ans=pd.DataFrame()

In [156]:
q = pd.read_csv("test.csv")

In [157]:
ans["shot_id_number"] = q["shot_id_number"]

In [158]:
ypreddf=pd.DataFrame(ypred)

In [159]:
ans["is_goal"]= ypreddf

In [160]:
ans.to_csv("final_prediction.csv",index=False)